In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import numpy as np

In [8]:
# Memastikan penggunaan GPU jika tersedia
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Device Name:", torch.cuda.get_device_name(0))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

CUDA Available: True
GPU Device Name: NVIDIA GeForce GTX 1050
Using device: cuda


In [9]:
# === 1. Setup Dataset ===
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

data = datasets.FashionMNIST(root="./data", train=True, download=True, transform=transform)
train_size = int(0.8 * len(data))
val_size = len(data) - train_size
train_data, val_data = random_split(data, [train_size, val_size])
test_data = datasets.FashionMNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [10]:
# Define CNN architecture
class CNN(nn.Module):
    def __init__(self, kernel_size=3, pooling='max'):
        super(CNN, self).__init__()
        
        if pooling == 'max':
            self.pool = nn.MaxPool2d(2, 2)
        elif pooling == 'avg':
            self.pool = nn.AvgPool2d(2, 2)

        self.conv1 = nn.Conv2d(1, 32, kernel_size=kernel_size, padding=kernel_size//2)  # Ganti 3 dengan 1
        self.conv2 = nn.Conv2d(32, 64, kernel_size=kernel_size, padding=kernel_size//2)
        self.fc1 = nn.Linear(64 * 7 * 7, 256)  # Sesuaikan ukuran input
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)  # Sesuaikan ukuran flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [11]:
# Training function with Early Stopping trigger
def train_model(model, optimizer, criterion, scheduler, num_epochs=5, early_stopping_patience=10):
    train_losses = []
    val_losses = []
    best_loss = float('inf')
    patience = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:  # Ganti trainloader dengan train_loader
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        train_loss = running_loss / len(train_loader)
        train_losses.append(train_loss)

        # Validation loss
        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for inputs, labels in val_loader:  # Ganti testloader dengan val_loader
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        val_losses.append(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # Early stopping trigger logic
        if val_loss < best_loss:
            best_loss = val_loss
            patience = 0
        else:
            patience += 1
            if patience >= early_stopping_patience:
                print("Early stopping triggered due to no improvement in validation loss.")
                break

        # Step the scheduler
        scheduler.step(val_loss)

    return train_losses, val_losses

In [12]:
# Compare models
kernel_sizes = [3, 5, 7]
pooling_methods = ['max', 'avg']
optimizers = {'SGD': optim.SGD, 'RMSProp': optim.RMSprop, 'Adam': optim.Adam}
num_epochs = [5, 50, 100, 250, 350]

results = {}

for kernel_size in kernel_sizes:
    for pooling in pooling_methods:
        for opt_name, opt_fn in optimizers.items():
            for epoch in num_epochs:
                print(f"Kernel: {kernel_size}, Pooling: {pooling}, Optimizer: {opt_name}, Epochs: {epoch}")

                model = CNN(kernel_size=kernel_size, pooling=pooling).to(device)
                criterion = nn.CrossEntropyLoss()
                optimizer = opt_fn(model.parameters(), lr=0.01)
                scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)

                train_losses, val_losses = train_model(
                    model, optimizer, criterion, scheduler, num_epochs=epoch, early_stopping_patience=10
                )

                # Save results
                results[(kernel_size, pooling, opt_name, epoch)] = (train_losses, val_losses)


Kernel: 3, Pooling: max, Optimizer: SGD, Epochs: 5
Epoch 1/5, Train Loss: 0.9474, Val Loss: 0.6012
Epoch 2/5, Train Loss: 0.5423, Val Loss: 0.5234
Epoch 3/5, Train Loss: 0.4672, Val Loss: 0.4503
Epoch 4/5, Train Loss: 0.4191, Val Loss: 0.4072
Epoch 5/5, Train Loss: 0.3884, Val Loss: 0.3802
Kernel: 3, Pooling: max, Optimizer: SGD, Epochs: 50
Epoch 1/50, Train Loss: 1.0165, Val Loss: 0.6218
Epoch 2/50, Train Loss: 0.5493, Val Loss: 0.5056
Epoch 3/50, Train Loss: 0.4736, Val Loss: 0.4574
Epoch 4/50, Train Loss: 0.4259, Val Loss: 0.4123
Epoch 5/50, Train Loss: 0.3944, Val Loss: 0.3877
Epoch 6/50, Train Loss: 0.3729, Val Loss: 0.3825
Epoch 7/50, Train Loss: 0.3549, Val Loss: 0.3639
Epoch 8/50, Train Loss: 0.3394, Val Loss: 0.3576
Epoch 9/50, Train Loss: 0.3251, Val Loss: 0.3401
Epoch 10/50, Train Loss: 0.3130, Val Loss: 0.3328
Epoch 11/50, Train Loss: 0.3029, Val Loss: 0.3183
Epoch 12/50, Train Loss: 0.2922, Val Loss: 0.3033
Epoch 13/50, Train Loss: 0.2839, Val Loss: 0.3078
Epoch 14/50, Tra

In [13]:
# Save results to a file
np.save('cnn_fashionmnist_results.npy', results)  # Ganti nama file untuk mencerminkan dataset
print("All experiments completed.")

best_config = None
best_val_loss = float('inf')

for config, (train_losses, val_losses) in results.items():
    min_val_loss = min(val_losses)
    if min_val_loss < best_val_loss:
        best_val_loss = min_val_loss
        best_config = config

print(f"Best Configuration: {best_config}, Best Validation Loss: {best_val_loss:.4f}")

All experiments completed.
Best Configuration: (5, 'max', 'SGD', 350), Best Validation Loss: 0.2343
